In [1]:
import os
import random
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.transformation import *
from utils.models import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.get_logger().setLevel(logging.ERROR)

catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")
mf_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")

2024-10-27 17:13:07.690764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-27 17:13:07.717781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-27 17:13:07.717824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 17:13:07.736706: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-27 17:13:09.184467: W tensorflow/compiler/tf

In [2]:
catalogo = calculate_ratio_of_interactions(mf_dataset, catalogo)

# Parámetros para el cálculo del puntaje
features = {"hito": (1, 4), "skill": (0, 15), "knowledge": (1, 15), "interaction_ratio": (0.0, 1.0)}
weights = {"hito": 0.3, "skill": 0.25, "knowledge": 0.25, "interaction_ratio": 0.2}

# Calcular el puntaje en el DataFrame
catalogo = calculate_score_dataset(catalogo, features, weights, new_column='formula')

scale_complexity_norm = MinMaxScaler()
catalogo['complexity_norm'] = scale_complexity_norm.fit_transform(catalogo[['complexity']])

scale_complexity12_norm = MinMaxScaler()
catalogo['complexity12_norm'] = scale_complexity12_norm.fit_transform(catalogo[['complexity12']])

df_catalogo = catalogo.copy()
df_catalogo.head()

,id_ejercicio,nombre,h1,h2,h3,h4,s1,s2,s3,s4,...,k4,hito,skill,knowledge,complexity,complexity12,interaction_ratio,formula,complexity_norm,complexity12_norm
0,0,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,...,1,1,1,7,23,279,0.787206,0.281251,0.111111,0.022774
1,1,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,...,1,2,1,1,17,529,0.359008,0.188468,0.080808,0.281573
2,2,Subsecuencias de ADN,0,0,1,0,0,0,1,1,...,1,2,3,7,55,567,0.022193,0.261581,0.272727,0.320911
3,3,NÃºmeros Primos,0,0,0,1,0,0,0,0,...,1,1,0,3,3,259,0.707572,0.177229,0.010101,0.002070
4,4,Nota Final,0,0,0,1,0,0,0,0,...,1,1,0,1,1,257,0.956919,0.191384,0.000000,0.000000


In [3]:
df_ratings = factorization_to_ratings(df_catalogo, mf_dataset, user_col='id_estudiante')
train_data, test_data = train_test_split(df_ratings, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
train_data.head(10)

,id_estudiante,id_ejercicio,nombre,h1,h2,h3,h4,s1,s2,s3,...,k4,hito,skill,knowledge,complexity,complexity12,interaction_ratio,formula,complexity_norm,complexity12_norm
1197,89,13,Cajero AutomÃ¡tico Nivel 2,0,0,0,1,0,0,0,...,1,1,1,3,19,275,0.304178,0.113216,0.090909,0.018634
8052,1099,22,Signo del ZodÃ­aco,0,0,0,1,0,0,0,...,1,1,1,1,17,273,0.848564,0.186379,0.080808,0.016563
5461,403,0,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,...,1,1,1,7,23,279,0.787206,0.281251,0.111111,0.022774
7389,1026,7,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.050914,0.000000,0.000000
6478,923,10,Descomponer un nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.781984,0.192111,0.010101,0.002070
1429,104,17,Suma de los N primeros nÃºmeros naturales,0,0,0,1,0,0,0,...,0,1,0,2,2,258,0.962141,0.210285,0.005051,0.001035
4464,336,7,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.050914,0.000000,0.000000
4092,307,4,Nota Final,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.956919,0.191384,0.000000,0.000000
2072,153,32,Cajero AutomÃ¡tico Nivel 1,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.515666,0.138847,0.010101,0.002070
8660,1159,18,Juego Adivina mi nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.507833,0.137281,0.010101,0.002070


In [4]:
Y_TRAIN = train_data['id_estudiante'].values
Y_TEST = test_data['id_estudiante'].values


## COMLEXITY (BINARY)
x_train_complexity_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'],axis=1).values
x_test_complexity_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'],axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## FORMULA (BINARY)
x_train_formula_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_formula_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values


## COMLEXITY (LABEL ENCODED)
x_train_complexity_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## FORMULA (BINARY)
x_train_formula_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_formula_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values

In [5]:
learning_rate = 0.001
epochs = 50
batch_size = 32
validation_split = 0.2
dropout_rate = 0.2
layer_units = [64, 32]

# Binary models

In [6]:
model_complexity_binary = TargetModel(input_shape=x_train_complexity_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_binary = model_complexity_binary.train(x_train_complexity_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_binary = model_complexity_binary.evaluate(x_test_complexity_binary, Y_TEST)
predictions_complexity_binary = model_complexity_binary.predict(x_test_complexity_binary)

Epoch 1/50


2024-10-27 17:13:12.898211: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 481093.9375 - val_loss: 362685.9688
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 364484.0000 - val_loss: 276975.9062
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 264865.5312 - val_loss: 192585.9062
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 198363.5312 - val_loss: 182034.7031
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 193705.6094 - val_loss: 181305.8594
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 193393.1875 - val_loss: 181474.8594
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 195603.0938 - val_loss: 180315.7344
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 194312.6094 - val_loss: 180075.6562
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 191159.7812 - val_loss: 179385.9375
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 188130.6250 - val_loss: 179365.5469
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/st

In [7]:
model_complexity12_binary = TargetModel(input_shape=x_train_complexity12_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_binary = model_complexity12_binary.train(x_train_complexity12_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_binary = model_complexity12_binary.evaluate(x_test_complexity12_binary, Y_TEST)
predictions_complexity12_binary = model_complexity12_binary.predict(x_test_complexity12_binary)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 336611.8438 - val_loss: 256835.6094
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 279606.8438 - val_loss: 255374.9062
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 268135.9062 - val_loss: 253772.4688
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 269676.6875 - val_loss: 251972.3750
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 266669.1875 - val_loss: 250499.4844
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 267711.3438 - val_loss: 246389.4219
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 263035.7188 - val_loss: 241995.0156
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 250032.3438 - val_loss: 234310.2344
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 246045.3438 - val_loss: 222260.5312
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 228331.3906 - val_loss: 208032.1562
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [8]:
model_formula_binary = TargetModel(input_shape=x_train_formula_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_formula_binary = model_formula_binary.train(x_train_formula_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_formula_binary = model_formula_binary.evaluate(x_test_formula_binary, Y_TEST)
predictions_formula_binary = model_formula_binary.predict(x_test_formula_binary)


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 492451.7188 - val_loss: 333880.6250
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 276552.8750 - val_loss: 189099.8906
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 195163.8281 - val_loss: 181405.0312
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 189598.0469 - val_loss: 179544.5625
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 192209.7812 - val_loss: 178826.2500
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 190823.0625 - val_loss: 178375.0000
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 193765.3906 - val_loss: 177952.8906
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 188537.0156 - val_loss: 177454.0625
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 182716.2969 - val_loss: 177240.2500
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 184700.0938 - val_loss: 176999.1875
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

# Labels models

In [9]:
model_complexity_label = TargetModel(input_shape=x_train_complexity_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_label = model_complexity_label.train(x_train_complexity_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_label = model_complexity_label.evaluate(x_test_complexity_label, Y_TEST)
predictions_complexity_label = model_complexity_label.predict(x_test_complexity_label)


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 484787.0312 - val_loss: 362919.6250
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 365273.1562 - val_loss: 301188.7812
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 306985.1250 - val_loss: 229348.1562
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 234006.8438 - val_loss: 206572.7969
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 225531.4688 - val_loss: 204038.1406
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 213536.9219 - val_loss: 201970.4844
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 209470.6250 - val_loss: 201043.8125
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 215317.3594 - val_loss: 199675.9062
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 214179.1406 - val_loss: 198126.9219
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 205426.5156 - val_loss: 196875.0312
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [10]:
model_complexity12_label = TargetModel(input_shape=x_train_complexity12_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_label = model_complexity12_label.train(x_train_complexity12_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_label = model_complexity12_label.evaluate(x_test_complexity12_label, Y_TEST)
predictions_complexity12_label = model_complexity12_label.predict(x_test_complexity12_label)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 358364.9062 - val_loss: 258000.6875
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 272602.9062 - val_loss: 261341.5781
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 278710.9375 - val_loss: 256685.5000
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 263749.7188 - val_loss: 256206.3594
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 272668.0312 - val_loss: 255413.3594
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 267679.0312 - val_loss: 254906.3125
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 271126.4375 - val_loss: 254582.0469
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 273514.7812 - val_loss: 253382.5781
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 269117.1562 - val_loss: 252125.6250
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 265809.5625 - val_loss: 251426.7500
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [11]:
model_formula_label = TargetModel(input_shape=x_train_formula_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_formula_label = model_formula_label.train(x_train_formula_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_formula_label = model_formula_label.evaluate(x_test_formula_label, Y_TEST)
predictions_formula_label = model_formula_label.predict(x_test_formula_label)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 507325.1875 - val_loss: 342124.4375
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 311675.2812 - val_loss: 245853.8750
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 263149.3438 - val_loss: 229619.7344
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 244503.9844 - val_loss: 216428.0469
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 227687.9375 - val_loss: 205115.2500
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 213860.9844 - val_loss: 197736.9219
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 209486.6250 - val_loss: 195085.5000
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 202978.7656 - val_loss: 193811.0156
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 201910.3281 - val_loss: 193046.4531
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 196664.7812 - val_loss: 192371.1406
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

# Results

### Estudiante

In [14]:
# 0 | 1305
rut_estudiante = 0
ejercicios_realizados = df_ratings[df_ratings['id_estudiante'] == rut_estudiante]['id_ejercicio'].values
ejercicios_no_realizados = catalogo[~catalogo['id_ejercicio'].isin(ejercicios_realizados)]
catalogo[catalogo['id_ejercicio'].isin(ejercicios_realizados)].sort_values(by='complexity12', ascending=True)[['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity
4,4,Nota Final,1,0,1,1
25,25,Ordenar tres nÃºmeros,1,0,1,1
17,17,Suma de los N primeros nÃºmeros naturales,1,0,2,2
23,23,Conversor de Decimal a Binario,1,0,2,2
3,3,NÃºmeros Primos,1,0,3,3
10,10,Descomponer un nÃºmero,1,0,3,3
18,18,Juego Adivina mi nÃºmero,1,0,3,3
26,26,Contestador AutomÃ¡tico,1,0,3,3
29,29,AprobaciÃ³n de CrÃ©ditos,1,0,3,3
42,42,NÃºmeros Amigos,1,0,3,3


### Resultados

In [15]:
predicciones_complexity_binary = model_complexity_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_binary = model_complexity12_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_formula_binary = model_formula_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


predicciones_complexity_label = model_complexity_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_label = model_complexity12_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_formula_label = model_formula_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [16]:
recomendations_complexity_binary = ejercicios_no_realizados.copy()
recomendations_complexity_binary['predicted_preference'] = predicciones_complexity_binary
recomendations_complexity_binary = recomendations_complexity_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['id_ejercicio'].isin(recomendations_complexity_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity
5,5,Validador de Expresiones MatemÃ¡ticas,4,8,7,135
7,7,Resolver un sistema de ecuaciones,1,0,1,1
13,13,Cajero AutomÃ¡tico Nivel 2,1,1,3,19
30,30,Interpretar Expresiones MatÃ©maticas,4,12,7,199
32,32,Cajero AutomÃ¡tico Nivel 1,1,0,3,3
35,35,Factores Primos,1,1,3,19
36,36,Pintar RegiÃ³n,4,8,7,135
43,43,Clase Usuario,3,3,15,63
47,47,El problema de la mochila,4,12,7,199
51,51,Combinaciones de NÃºmeros Binarios,4,12,7,199


In [17]:
recomendations_complexity12_binary = ejercicios_no_realizados.copy()
recomendations_complexity12_binary['predicted_preference'] = predicciones_complexity12_binary
recomendations_complexity12_binary = recomendations_complexity12_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['id_ejercicio'].isin(recomendations_complexity12_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity12']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity12
5,5,Validador de Expresiones MatemÃ¡ticas,4,8,7,1159
7,7,Resolver un sistema de ecuaciones,1,0,1,257
13,13,Cajero AutomÃ¡tico Nivel 2,1,1,3,275
30,30,Interpretar Expresiones MatÃ©maticas,4,12,7,1223
32,32,Cajero AutomÃ¡tico Nivel 1,1,0,3,259
35,35,Factores Primos,1,1,3,275
36,36,Pintar RegiÃ³n,4,8,7,1159
47,47,El problema de la mochila,4,12,7,1223
50,50,MCM recursivo,4,1,3,1043
51,51,Combinaciones de NÃºmeros Binarios,4,12,7,1223


In [18]:
recomendations_formula_binary = ejercicios_no_realizados.copy()
recomendations_formula_binary['predicted_preference'] = predicciones_formula_binary
recomendations_formula_binary = recomendations_formula_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['id_ejercicio'].isin(recomendations_formula_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'formula']]

,id_ejercicio,nombre,hito,skill,knowledge,formula
14,14,Clase Auto,3,2,11,0.411905
16,16,OrnitologÃ­a 1,3,0,8,0.325000
19,19,Cuenta Corriente en un Banco,3,2,11,0.411905
34,34,OrnitologÃ­a 3,3,0,12,0.396429
41,41,Clase FechaHora,3,2,15,0.483333
43,43,Clase Usuario,3,3,15,0.500000
45,45,Carro de Compras,3,2,15,0.483333
46,46,Pokemon,3,3,15,0.500000
49,49,Caminos entre ciudades,3,2,15,0.483333
52,52,OrnitologÃ­a 2,3,0,8,0.325000
